In [3]:
import cv2
import numpy as np
import os

# --- Helper: extract trajectory without showing any window ---
def extract_trajectory(video_path, lower=(0, 50, 50), upper=(10, 255, 255)):
    cap = cv2.VideoCapture(video_path)
    positions = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        lower_np = np.array(lower, dtype=np.uint8)
        upper_np = np.array(upper, dtype=np.uint8)
        mask = cv2.inRange(hsv, lower_np, upper_np)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            c = max(contours, key=cv2.contourArea)
            (x, y, w, h) = cv2.boundingRect(c)
            positions.append((x + w // 2, y + h // 2))
    cap.release()
    return positions

# --- Helper: draw trajectory to image ---
def trajectory_to_image(positions, size=(400, 400)):
    img = np.zeros(size + (3,), dtype=np.uint8)
    for i in range(1, len(positions)):
        cv2.line(img, positions[i - 1], positions[i], (255, 255, 255), 3)
    return img

# --- Helper: match letter shape ---
def compare_with_letters(img):
    letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    scores = {}
    for ch in letters:
        temp = np.zeros_like(img)
        cv2.putText(temp, ch, (100, 300), cv2.FONT_HERSHEY_SIMPLEX, 6, (255, 255, 255), 15)
        score = cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
                                  cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY),
                                  cv2.TM_CCOEFF_NORMED)[0][0]
        scores[ch] = score
    return max(scores, key=scores.get), max(scores.values())

# --- MAIN EXECUTION ---
video_dir = "videos"
flag_letters = []

for i in range(1, 7):
    path = os.path.join(video_dir, f"char_{i}.mp4")
    if not os.path.exists(path):
        print(f"❌ Missing: {path}")
        continue
    positions = extract_trajectory(path)
    traj_img = trajectory_to_image(positions)
    letter, score = compare_with_letters(traj_img)
    flag_letters.append(letter)
    print(f"✅ {os.path.basename(path)} → {letter} (score={score:.3f})")

flag = "FLAG{" + "".join(flag_letters) + "}"
print("\n🏁 Final FLAG:", flag)

✅ char_1.mp4 → T (score=0.077)
✅ char_2.mp4 → M (score=0.110)
✅ char_3.mp4 → A (score=0.064)
✅ char_4.mp4 → W (score=0.114)
✅ char_5.mp4 → Q (score=0.091)
✅ char_6.mp4 → A (score=0.092)

🏁 Final FLAG: FLAG{TMAWQA}
